In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


# New Section

In [3]:
from datasets import load_dataset

In [4]:
sms_dataset = load_dataset("sms_spam")
sms_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for sms_spam contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sms_spam
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [5]:
import pandas as pd
from datasets import load_dataset

texts = sms_dataset["train"]["sms"]
labels = sms_dataset["train"]["label"]

train_set = pd.DataFrame({'text': texts, 'label': labels})

train_set['text'] = train_set['text'].str.strip()

# Sostituisci 0/1 in "label" con "not spam" e "spam"
train_set['label'] = train_set['label'].replace({1: 'spam', 0: 'not spam'})

# Rinomina la colonna "label" in "annotation"
train_set = train_set.rename(columns={'label': 'annotation'})

# Salva il DataFrame in un file CSV
train_set.to_csv('train_set.csv', index=False)


In [8]:
# test set (preso dall'excel)
test_set = pd.read_csv('test-set(excel).csv')#, sep='\t')

print(test_set)

                                                   text annotation
0                No message..no responce..what happend?   not spam
1     You are everywhere dirt, on the floor, the win...   not spam
2     Pls tell nelson that the bb's are no longer co...   not spam
3     Get ur 1st RINGTONE FREE NOW! Reply to this ms...       spam
4                                        See you there!   not spam
...                                                 ...        ...
1110         You still around? Looking to pick up later   not spam
1111         Alright we're hooked up, where you guys at   not spam
1112  It's ok i noe u're busy but i'm really too bor...   not spam
1113  Good day to You too.Pray for me.Remove the tee...   not spam
1114  Spook up your mob with a Halloween collection ...       spam

[1115 rows x 2 columns]


In [9]:
# Filtra il DataFrame mantenendo solo le righe non presenti in text (da excel)
train_filter = train_set[~train_set['text'].isin(test_set['text'])]

print(train_filter)

print(len(train_filter))
print(len(train_set))
print(len(test_set))

                                                   text annotation
0     Go until jurong point, crazy.. Available only ...   not spam
1                         Ok lar... Joking wif u oni...   not spam
3     U dun say so early hor... U c already then say...   not spam
8     WINNER!! As a valued network customer you have...       spam
9     Had your mobile 11 months or more? U R entitle...       spam
...                                                 ...        ...
5568  REMINDER FROM O2: To get 2.50 pounds free call...       spam
5570               Will ü b going to esplanade fr home?   not spam
5571  Pity, * was in mood for that. So...any other s...   not spam
5572  The guy did some bitching but I acted like i'd...   not spam
5573                         Rofl. Its true to its name   not spam

[4343 rows x 2 columns]
4343
5574
1115


In [11]:
import random

# 5 frasi random not spam
random_sentences_0 = train_filter[train_filter['annotation'] == "not spam"]['text'].sample(10).reset_index(drop=True)

# 5 frasi random spam
random_sentences_1 = train_filter[train_filter['annotation'] == "spam"]['text'].sample(10).reset_index(drop=True)


sentences_group_0 = list(random_sentences_0)
sentences_group_1 = list(random_sentences_1)

In [12]:
#### nuovo prompt no lista

# nuove colonne per i prompt
for n in [10, 8, 6, 4, 2]:
    test_set[f'prompt{n}'] = (
        'Which one of the attributes: "spam", "not spam" describes a given text?\n'
        'Answer should be only a string. Possible value "spam" or "not spam"\n\n'
        'These are examples of "spam":\n' +
        "\n".join(f"- {sentence}" for sentence in random_sentences_1[:n]) +
        '\n\nThese are examples of "not spam":\n' +
        "\n".join(f"- {sentence}" for sentence in random_sentences_0[:n]) +
        "\n\nContent of the SMS: " + test_set['text']
    )

print(test_set.head(10))

                                                text annotation  \
0             No message..no responce..what happend?   not spam   
1  You are everywhere dirt, on the floor, the win...   not spam   
2  Pls tell nelson that the bb's are no longer co...   not spam   
3  Get ur 1st RINGTONE FREE NOW! Reply to this ms...       spam   
4                                     See you there!   not spam   
5     Great. So should i send you my account number.   not spam   
6  WIN: We have a winner! Mr. T. Foley won an iPo...       spam   
7                Do we have any spare power supplies   not spam   
8  Ok good then i later come find ü... C lucky i ...   not spam   
9                   Sorry for the delay. Yes masters   not spam   

                                            prompt10  \
0  Which one of the attributes: "spam", "not spam...   
1  Which one of the attributes: "spam", "not spam...   
2  Which one of the attributes: "spam", "not spam...   
3  Which one of the attributes: "spam"

In [ ]:
# test su csv
test_set.head(10).to_csv("output_file.csv", index=False)

In [13]:
# colonne per csv totale
selected_columns = ["text", "annotation", "prompt10", "prompt8", "prompt6", "prompt4", "prompt2"]

# Salva il DataFrame con solo le colonne selezionate in un file CSV
test_set[selected_columns].to_csv("06_Spam.csv", index=False)

In [14]:
len(test_set[selected_columns])

1115